### Introduction to `MuJoCo`
```
MuJoCo is a free and open source physics engine that aims to facilitate research and development in robotics, biomechanics, graphics and animation, and other areas where fast and accurate simulation is needed.
```
- `MuJoCo` stands for `Mu`lti-`Jo`int dynamics with `Co`ntact
- It is a (rigid-body) simulator
- In my personal oppinion, a (proper) simulator consists of
    - Rigid-body simulation by solving an equation of motion (e.g., an articulated body algorithm)
    - Contact solver (main cause of sim-to-real gap)
    - Easy-to-use visualizer (`VERY IMPORTANT` for conducting research)
 
##### Throughout the tutorial following topics will be covered:
1. Introduction to `MuJoCo` and our lab-maintained mujoco_parser
2. Forward Kinematics
3. Forward Dynamics
4. Inverse Kinematics
5. Inverse Dynamics
6. Planning using `RRT`
7. Vision Reasoning using `GPT4o` and open-vocab detection
8. Motion Retargeting of Unitree `G1`
9. PID Control of `Snapbot`
10. Reinforcement Learning using `Soft Actor-Critic` of `Snapbot`

# Enjoy!

In [2]:
!pip show mujoco

Name: mujoco
Version: 3.1.1
Summary: MuJoCo Physics Simulator
Home-page: https://github.com/google-deepmind/mujoco
Author: 
Author-email: Google DeepMind <mujoco@deepmind.com>
License: Apache License 2.0
Location: /Users/sb/miniconda3/envs/3.11/lib/python3.11/site-packages
Requires: absl-py, etils, glfw, numpy, pyopengl
Required-by: mujoco-python-viewer


In [3]:
import sys,mujoco
sys.path.append('../package/helper/')
sys.path.append('../package/mujoco_usage/')
from mujoco_parser import *
from utility import *
print ("MuJoCo:[%s]"%(mujoco.__version__))

MuJoCo:[3.1.1]


#### Parse environment

In [ ]:
xml_path = '../asset/object/floor_isaac_style.xml'
env = MuJoCoParserClass(name='Floor',rel_xml_path=xml_path,verbose=True)

#### Render `floor`

In [ ]:
env.reset(step=True)
env.init_viewer()
while env.is_viewer_alive():
    env.step()
    env.render()
print ("Done.")

#### Render with other plot functions

In [ ]:
import numpy as np

env.reset(step=True)
env.init_viewer()
while env.is_viewer_alive():
    env.step()
    # Render
    env.plot_T(print_xyz=True)
    env.plot_time()
    env.plot_ellipsoid(p=np.array([0,-2.0,1.0]),rx=0.1,ry=0.2,rz=0.3,rgba=(1,0,1,0.5))
    env.plot_cylinder(p=np.array([0,-1.0,1.0]),r=0.1,h=0.2,rgba=(1,1,0,0.5))
    env.plot_T(p=np.array([0,0,1.0]),axis_len=0.2,axis_width=0.01)
    env.plot_arrow(p=np.array([0,1.0,1.0]),R=np.eye(3),r=0.1,h=0.5,rgba=(1,0,0,0.5))
    env.plot_box(p=np.array([0,2.0,1.0]),R=np.eye(3),xlen=0.2,ylen=0.2,zlen=0.1,rgba=(0,1,0,0.5))
    env.plot_capsule(p=np.array([0,3.0,1.0]),R=np.eye(3),r=0.1,h=0.1,rgba=(0,0,1,0.5))
    env.render()
print ("Done.")

#### Parse `Unitree G1`

In [ ]:
xml_path = '../asset/unitree_g1/scene_g1.xml'
env = MuJoCoParserClass(name='Unitree G1',rel_xml_path=xml_path,verbose=True)

#### Render `G1`

In [ ]:
env.reset(step=True)
env.init_viewer()
while env.is_viewer_alive():
    env.step()
    env.render()
print ("Done.")

#### Render `G1` with other information (contact, joint axis, etc..)

In [ ]:
env.reset(step=True)
env.init_viewer(transparent=True)
while env.is_viewer_alive():
    env.step()
    if env.loop_every(tick_every=10):
        env.plot_T()
        env.plot_time() # time
        env.plot_contact_info() # contact information
        env.plot_joint_axis(axis_len=0.025,axis_r=0.005) # revolute joints
        env.plot_links_between_bodies(rgba=(0,0,0,1),r=0.001) # link information
        env.render()
print ("Done.")

#### Parse `RB-Y1`

In [4]:
xml_path = '../asset/rby/scene_rby.xml'
env = MuJoCoParserClass(name='RB-Y1',rel_xml_path=xml_path,verbose=True)

name:[RB-Y1] dt:[0.005] HZ:[200]
n_qpos:[33] n_qvel:[32] n_qacc:[32] n_ctrl:[24]

n_body:[28]
 [0/28] [world] mass:[0.00]kg
 [1/28] [mobile_base] mass:[130.00]kg
 [2/28] [link1] mass:[30.00]kg
 [3/28] [link2] mass:[20.00]kg
 [4/28] [link3] mass:[3.00]kg
 [5/28] [link4] mass:[1.00]kg
 [6/28] [link5] mass:[3.00]kg
 [7/28] [link6] mass:[5.00]kg
 [8/28] [left_arm_1] mass:[1.00]kg
 [9/28] [left_arm_2] mass:[1.00]kg
 [10/28] [left_arm_3] mass:[1.00]kg
 [11/28] [left_arm_4] mass:[1.00]kg
 [12/28] [left_arm_5] mass:[0.50]kg
 [13/28] [left_arm_6] mass:[0.50]kg
 [14/28] [left_arm_7] mass:[0.10]kg
 [15/28] [left_arm_f1] mass:[0.01]kg
 [16/28] [left_arm_f2] mass:[0.01]kg
 [17/28] [right_arm_1] mass:[1.00]kg
 [18/28] [right_arm_2] mass:[1.00]kg
 [19/28] [right_arm_3] mass:[1.00]kg
 [20/28] [right_arm_4] mass:[1.00]kg
 [21/28] [right_arm_5] mass:[0.50]kg
 [22/28] [right_arm_6] mass:[0.50]kg
 [23/28] [right_arm_7] mass:[0.10]kg
 [24/28] [right_arm_f1] mass:[0.01]kg
 [25/28] [right_arm_f2] mass:[0.01]

#### Render `RB-Y1`

In [5]:
env.reset(step=True)
env.init_viewer(transparent=True)
while env.is_viewer_alive():
    env.step()
    if env.loop_every(tick_every=10):
        env.plot_T()
        env.plot_time() # time
        env.plot_contact_info() # contact information
        env.render()
print ("Done.")

Done.


#### Render `Google Scanned Objects`

In [6]:
xml_path = '../asset/scanned_objects/scene_objects.xml'
env = MuJoCoParserClass(name='Scanned Objects',rel_xml_path=xml_path,verbose=True)

name:[Scanned Objects] dt:[0.002] HZ:[500]
n_qpos:[49] n_qvel:[42] n_qacc:[42] n_ctrl:[0]

n_body:[9]
 [0/9] [world] mass:[0.00]kg
 [1/9] [base_table] mass:[1.00]kg
 [2/9] [model_3D_Dollhouse_Sink] mass:[1.20]kg
 [3/9] [model_3D_Dollhouse_Sofa] mass:[1.81]kg
 [4/9] [model_3D_Dollhouse_Swing] mass:[0.53]kg
 [5/9] [model_3D_Dollhouse_TablePurple] mass:[0.21]kg
 [6/9] [Android_Figure_Chrome] mass:[0.22]kg
 [7/9] [Android_Figure_Orange] mass:[0.22]kg
 [8/9] [Android_Figure_Panda] mass:[0.22]kg
body_total_mass:[5.42]kg

n_geom:[233]
geom_names:['floor', 'base_table', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None,

In [8]:
env.reset(step=True)
# Set object positions
obj_names = env.get_body_names(excluding='world') # object names
n_obj = len(obj_names)
obj_xyzs = sample_xyzs(
    n_obj,
    x_range   = [-0.45,+0.45],
    y_range   = [-0.45,+0.45],
    z_range   = [0.51,0.51],
    min_dist  = 0.2,
    xy_margin = 0.0
)
for obj_idx in range(n_obj):
    env.set_p_base_body(body_name=obj_names[obj_idx],p=obj_xyzs[obj_idx,:])
    env.set_R_base_body(body_name=obj_names[obj_idx],R=np.eye(3,3))
# Loop
env.init_viewer(transparent=False)
while env.is_viewer_alive():
    env.step()
    if env.loop_every(tick_every=10):
        env.plot_T()
        env.plot_time() # time
        env.plot_contact_info() # contact information
        env.render()
print ("Done.")


Done.


: 